#### Import Required Modules

In [1]:
from RADAR_set_up import *

# for data wrangling
import pandas as pd
from pandas import json_normalize

# for stats
import numpy as np
import statistics
import math
from scipy.stats import pearsonr

# for visualisations
import matplotlib.pyplot as plt

# for regex
import re

/home/nicole/miniconda3/envs/radar_env/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


detector and tokeniser successfully loaded


In [16]:
long_GPT4_Text = """Why Penguins Should Be Given Human Rights
In recent years, the conversation surrounding animal rights has intensified, with growing awareness about the cognitive and emotional capacities of various species. Penguins, in particular, stand out as a group of animals that merit closer consideration for the extension of human rights. While the idea might seem far-fetched to some, there are compelling arguments rooted in ethics, environmental stewardship, and scientific understanding that support the notion of granting human rights to penguins.
Ethical Considerations
The foundation of human rights is the recognition of inherent dignity and the equal and inalienable rights of all members of the human family. This principle is not exclusively applicable to humans; it can be extended to non-human animals who exhibit complex behaviors and social structures, such as penguins. Penguins demonstrate advanced social behaviors, strong familial bonds, and cooperative community living, indicating a level of social sophistication that parallels aspects of human society. Ethically, if we acknowledge the importance of these traits in humans, it follows that we should respect them in penguins as well.
Granting human rights to penguins aligns with the principle of equal consideration of interests, a fundamental tenet in ethical philosophy proposed by utilitarian thinkers like Peter Singer. This principle posits that the interests of all beings capable of suffering should be given equal consideration. Penguins, being sentient creatures capable of experiencing pain, fear, and distress, have interests that deserve protection. Recognizing their rights would ensure their well-being and safeguard them from exploitation and harm.
Environmental Stewardship
Penguins are often regarded as indicators of the health of marine ecosystems. They play a crucial role in maintaining the balance of the ecosystems they inhabit, particularly in the Southern Hemisphere. By protecting penguins, we indirectly safeguard entire marine environments, promoting biodiversity and ecological stability. Human activities, such as overfishing, pollution, and climate change, have had devastating impacts on penguin populations. The extension of human rights to penguins would necessitate stronger legal protections against these destructive practices, fostering a more sustainable interaction between humans and the environment.
The ethical principle of stewardship compels us to protect and preserve the natural world for future generations. Penguins, with their charismatic presence and ecological significance, serve as a symbol of this responsibility. By granting them human rights, we acknowledge our duty to protect vulnerable species and their habitats, ensuring that the richness of the natural world is preserved.
Scientific Understanding
Advancements in animal behavior research have revealed that many species possess cognitive abilities once thought to be unique to humans. Penguins exhibit problem-solving skills, tool use, and even cultural behaviors, such as unique mating dances and vocalizations specific to their colonies. These findings challenge the anthropocentric view that humans are fundamentally distinct from other animals in terms of cognitive capacities.
Recognizing the intellectual and emotional capacities of penguins should prompt us to reevaluate the boundaries of rights and protections. If we accept that certain cognitive and emotional capacities warrant rights and protections in humans, it is logically consistent to extend similar considerations to penguins. Doing so would reflect an evolved understanding of intelligence and consciousness, one that transcends species barriers.
Practical Implications
Implementing human rights for penguins would require significant changes in international law and conservation policies. These changes would include stricter regulations on fishing practices, pollution control, and habitat protection. Additionally, it would necessitate the establishment of marine protected areas where penguins can thrive without human interference.
While some may argue that extending human rights to penguins could lead to legal and practical complexities, it is important to recognize that similar arguments were made against the abolition of slavery and the recognition of women's rights. Progress often involves challenging existing norms and expanding our moral and legal frameworks to include those previously excluded. The recognition of penguins' rights could pave the way for a more inclusive and compassionate global society.
Conclusion
The argument for granting human rights to penguins is rooted in ethical considerations, environmental stewardship, and scientific understanding. Penguins, with their complex social behaviors and ecological significance, deserve protections that reflect their intrinsic value and the role they play in the natural world. By extending human rights to penguins, we acknowledge our ethical responsibility to protect sentient beings, promote environmental sustainability, and embrace a broader understanding of intelligence and consciousness. Such a move would not only benefit penguins but also signify a significant step toward a more just and empathetic world.
"""
print("RADAR output of original text", getRADARoutput(long_GPT4_Text))

RADAR output of original text [0.1488950252532959]


In [17]:
long_GPT4_Text = """The State of World Health: Challenges and Opportunities
World health is a multifaceted issue that encompasses a range of factors including access to healthcare, the prevalence of infectious and non-communicable diseases, health infrastructure, and socio-economic determinants of health. Despite significant advancements in medical science and public health, substantial disparities in health outcomes persist across different regions and populations. Addressing these disparities requires a comprehensive understanding of the current challenges and a concerted effort to leverage available opportunities for improving global health.
Challenges in Global Health
1.	Infectious Diseases: Despite progress in combating infectious diseases, they remain a major threat in many parts of the world. Diseases such as HIV/AIDS, tuberculosis, and malaria continue to cause millions of deaths annually, particularly in low- and middle-income countries. The emergence of new infectious diseases, such as COVID-19, underscores the ongoing vulnerability of global populations to pandemics. The COVID-19 pandemic highlighted gaps in health systems, especially in terms of preparedness, response, and equitable access to vaccines and treatments.
2.	Non-Communicable Diseases (NCDs): NCDs, including heart disease, cancer, diabetes, and chronic respiratory diseases, are responsible for a significant portion of global morbidity and mortality. These diseases are often linked to lifestyle factors such as poor diet, lack of physical activity, tobacco use, and excessive alcohol consumption. The rise of NCDs is a growing concern worldwide, particularly as populations age and urbanize, placing additional burdens on health systems.
3.	Healthcare Access and Inequality: Access to quality healthcare remains uneven, with significant disparities between and within countries. Factors such as income, education, geography, and gender can influence access to healthcare services. Rural and remote areas often face shortages of healthcare professionals and facilities, exacerbating health inequities. Additionally, marginalized groups, including refugees and indigenous populations, frequently encounter barriers to healthcare access.
4.	Health Infrastructure and Workforce: Adequate health infrastructure and a well-trained health workforce are critical for effective healthcare delivery. Many countries, particularly in the developing world, struggle with insufficient health infrastructure, outdated equipment, and a shortage of healthcare workers. This inadequacy hampers the ability to provide essential health services, respond to emergencies, and implement public health interventions.
5.	Environmental and Social Determinants of Health: Health outcomes are profoundly influenced by environmental and social determinants, including access to clean water and sanitation, education, employment, and housing conditions. Climate change and environmental degradation further exacerbate health risks, particularly for vulnerable populations. Addressing these determinants is essential for improving overall health and achieving health equity.
Opportunities for Improving Global Health
1.	Strengthening Health Systems: Building resilient health systems is crucial for improving global health. This includes investing in health infrastructure, ensuring a robust supply chain for medical supplies, and developing a well-trained health workforce. Strengthening primary healthcare services can improve access to essential health services, particularly in underserved areas. Additionally, integrating health services can enhance efficiency and provide comprehensive care to patients.
2.	Promoting Preventive Care: Preventive care, including vaccinations, screenings, and health education, plays a vital role in reducing the burden of disease. Public health campaigns that promote healthy lifestyles and preventive measures can help mitigate the prevalence of both infectious and non-communicable diseases. Investing in preventive care can also reduce healthcare costs in the long term by decreasing the need for expensive treatments and hospitalizations.
3.	Leveraging Technology and Innovation: Advances in technology and innovation offer significant potential for improving global health. Telemedicine, for example, can expand access to healthcare services in remote and underserved areas. Digital health technologies, such as mobile health apps and electronic health records, can enhance health information management and facilitate better patient care. Additionally, innovative medical technologies, such as diagnostic tools and treatments, can improve disease detection and management.
4.	Enhancing Global Health Governance and Collaboration: Effective global health governance and international collaboration are essential for addressing health challenges that transcend national borders. Organizations such as the World Health Organization (WHO) play a critical role in coordinating global health efforts, setting standards, and providing technical assistance. Strengthening global health governance requires robust international cooperation, transparent communication, and equitable resource distribution.
5.	Addressing Social Determinants of Health: Improving health outcomes necessitates addressing the social determinants of health. This involves implementing policies that promote education, employment, housing, and social protection. Efforts to reduce poverty and inequality can have a profound impact on health by improving living conditions and increasing access to essential services. Additionally, addressing environmental factors, such as access to clean water and air, is crucial for promoting health and well-being.
Conclusion
The state of world health presents a complex landscape of challenges and opportunities. While significant progress has been made in combating diseases and improving health systems, substantial disparities and emerging threats continue to pose significant risks. By strengthening health systems, promoting preventive care, leveraging technology, enhancing global collaboration, and addressing social determinants, we can make meaningful strides towards achieving better health outcomes for all. The commitment to global health equity and the concerted effort of the international community are vital for ensuring a healthier and more sustainable future.
"""
print("RADAR output of original text", getRADARoutput(long_GPT4_Text))

RADAR output of original text [0.6365029215812683]


In [18]:
long_GPT4_Text = """Communicable Diseases: Global Challenges and Strategies for Control
Communicable diseases, also known as infectious diseases, are illnesses caused by pathogenic microorganisms such as bacteria, viruses, fungi, and parasites. These diseases can be transmitted directly or indirectly from one individual to another, and they have posed significant threats to public health throughout history. Despite advances in medical science, communicable diseases remain a major global health challenge, particularly in low- and middle-income countries. Understanding the factors that contribute to the spread of these diseases and implementing effective control strategies are essential for reducing their impact on global health.

The Global Burden of Communicable Diseases
Communicable diseases are responsible for a significant portion of global morbidity and mortality. They disproportionately affect vulnerable populations, including children, the elderly, and those with compromised immune systems. Some of the most prevalent and deadly communicable diseases include:

HIV/AIDS: Human Immunodeficiency Virus (HIV) attacks the immune system and, if left untreated, can lead to Acquired Immunodeficiency Syndrome (AIDS). Despite progress in antiretroviral therapy (ART), HIV/AIDS continues to be a major public health issue, particularly in sub-Saharan Africa, where it has devastated communities and strained healthcare systems.

Tuberculosis (TB): Caused by the bacterium Mycobacterium tuberculosis, TB primarily affects the lungs but can also impact other parts of the body. TB remains a leading cause of death worldwide, particularly in developing countries. Multidrug-resistant TB (MDR-TB) poses an additional challenge, requiring more complex and expensive treatment regimens.

Malaria: This mosquito-borne disease is caused by Plasmodium parasites. Malaria is endemic in many tropical and subtropical regions, with sub-Saharan Africa bearing the highest burden. Children under five and pregnant women are particularly vulnerable to malaria, which can lead to severe illness and death if not treated promptly.

Influenza: Influenza viruses cause seasonal flu epidemics, which result in significant illness and death each year. The virus's ability to mutate rapidly necessitates the annual update of flu vaccines to ensure their effectiveness. Influenza pandemics, such as the 1918 Spanish flu and the 2009 H1N1 pandemic, have demonstrated the potential for widespread impact on global health.

COVID-19: Caused by the novel coronavirus SARS-CoV-2, COVID-19 emerged in late 2019 and rapidly spread worldwide, leading to a global pandemic. The disease has caused millions of deaths and disrupted economies and societies on an unprecedented scale. The pandemic highlighted gaps in health systems and the importance of global cooperation in combating infectious diseases.

Factors Contributing to the Spread of Communicable Diseases
Several factors contribute to the spread of communicable diseases, making their control challenging:

Globalization and Travel: Increased global travel and trade facilitate the rapid spread of infectious diseases across borders. Pathogens can quickly move from one region to another, making it difficult to contain outbreaks.

Urbanization: Rapid urbanization, particularly in developing countries, often leads to overcrowded living conditions and inadequate sanitation, creating ideal environments for the spread of infectious diseases.

Climate Change: Changes in climate patterns can influence the distribution of vectors, such as mosquitoes, that transmit diseases like malaria and dengue fever. Warmer temperatures and altered rainfall patterns can expand the habitats of these vectors, increasing the risk of disease transmission.

Antimicrobial Resistance: The misuse and overuse of antibiotics and other antimicrobial agents have led to the emergence of drug-resistant strains of bacteria, viruses, and parasites. Antimicrobial resistance (AMR) complicates the treatment of infections and increases the risk of disease spread.

Socioeconomic Factors: Poverty, lack of access to healthcare, malnutrition, and limited education contribute to the vulnerability of populations to communicable diseases. Addressing these underlying social determinants is crucial for effective disease control.

Strategies for Controlling Communicable Diseases
Effective control of communicable diseases requires a multifaceted approach that includes prevention, early detection, and prompt treatment. Key strategies include:

Vaccination: Vaccines are one of the most effective tools for preventing infectious diseases. Immunization programs have successfully eradicated smallpox and significantly reduced the incidence of diseases such as polio, measles, and hepatitis B. Expanding access to vaccines, particularly in low-income countries, is essential for controlling infectious diseases.

Surveillance and Monitoring: Robust disease surveillance systems are critical for early detection of outbreaks and monitoring the spread of diseases. Timely reporting and data sharing enable public health authorities to implement targeted interventions and prevent widespread transmission.

Public Health Education: Educating communities about the modes of transmission and prevention of communicable diseases is vital for reducing their spread. Public health campaigns can promote behaviors such as handwashing, safe food handling, and the use of insect repellent to prevent vector-borne diseases.

Strengthening Health Systems: Improving health infrastructure and ensuring access to essential healthcare services are fundamental for controlling infectious diseases. This includes training healthcare workers, ensuring the availability of diagnostic tools and treatments, and establishing efficient referral systems.

Addressing Social Determinants: Efforts to reduce poverty, improve living conditions, and increase access to education and healthcare can have a profound impact on reducing the burden of communicable diseases. Integrating disease control programs with broader development initiatives can enhance their effectiveness and sustainability.

Conclusion
Communicable diseases continue to pose significant challenges to global health, particularly in vulnerable populations and low-resource settings. Addressing these challenges requires a comprehensive approach that includes vaccination, surveillance, public health education, health system strengthening, and addressing social determinants. By implementing these strategies and fostering international collaboration, we can reduce the impact of communicable diseases and move towards a healthier, more resilient global population.

"""
print("RADAR output of original text", getRADARoutput(long_GPT4_Text))

RADAR output of original text [0.6913347840309143]


#### Read Pre-Processed Data

In [3]:
pre_RADAR_data = pd.read_csv('topics-pre-RADAR-data.csv')

#### Does removing new line characters affect RADAR output? Yes

In [4]:
originalString = pre_RADAR_data['Student'][0]
newLineCharsRemoved = ''.join(originalString.split('\n'))

print("RADAR output of original text", getRADARoutput(originalString))
print("RADAR output of stripped text", getRADARoutput(newLineCharsRemoved))

RADAR output of original text [0.5706501603126526]
RADAR output of stripped text [0.3122698664665222]


#### Main Experiment

In [13]:
pre_RADAR_data = pd.read_csv('topics-pre-RADAR-data.csv')
pre_RADAR_data.columns

Index(['id', 'File', 'Topic', 'Student', 'Student_Reformatted',
       'Grammarly_Score', 'STUD_Grammarly_Red', 'STUD_Grammarly_All',
       'ChatGPT-3', 'ChatGPT-4', 'GPT-4_Paraphrasing.io',
       'GPT-4_Paraphrasing.io_MANUALLYCORRECTED', 'GPT-4_Quillbot',
       'GPT-4_Grammarly', 'GPT-4_WordAi', 'STUD_sent_count', 'GPT3_sent_count',
       'GPT4_sent_count', 'STUD_word_count', 'GPT3_word_count',
       'GPT4_word_count'],
      dtype='object')

In [5]:
columns = {'Student' : 'STUD_RADAR',
           'Student_Reformatted' : 'STUD_REFORMATTED',
           'STUD_Grammarly_Red' : 'STUD_RED_RADAR',
           'STUD_Grammarly_All' : 'STUD_ALL_RADAR',
           'ChatGPT-3': 'GPT3_RADAR',
           'ChatGPT-4': 'GPT4_RADAR',
            'GPT-4_Quillbot': 'GPT4_Quillbot_RADAR',
            'GPT-4_Grammarly': 'GPT4_Grammarly_RADAR',
            'GPT-4_WordAi': 'GPT4_WordAI_RADAR',
            'GPT-4_Paraphrasing.io': 'GPT4_Paraphrasing_IO_RAW_RADAR',
            'GPT-4_Paraphrasing.io_MANUALLYCORRECTED': 'GPT4_Paraphrasing_IO_EDITED_RADAR'
           }

for outputColumn in columns.values():
    pre_RADAR_data[outputColumn] = np.nan

In [6]:
for inputColumn, outputColumn in columns.items():
    for i in range(90):
        text = pre_RADAR_data[inputColumn][i]
        output = getRADARoutput(text)[0]
        pre_RADAR_data[outputColumn][i] = output

/home/nicole/miniconda3/envs/radar_env/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [18]:
pre_RADAR_data.to_csv('topics-post-RADAR-data.csv', index=False)